In [35]:
import pandas as pd
import requests
import plotly.express as px

In [36]:
# Use the correct base URL for Binance Futures API (fapi.binance.com)
url = f"https://fapi.binance.com/fapi/v1/klines"
params = {
    "symbol": "SOLUSDT",
    "interval": "1m",
    "limit": 100
}

try:
    response = requests.get(url, params=params)
    response.raise_for_status() # Raises an HTTPError for bad responses (4xx or 5xx)
    data = response.json()
    print(data)
except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")
    if 'response' in locals():
        print(f"Response text: {response.text}")

[[1764525240000, '138.3600', '138.3800', '138.3400', '138.3600', '2951.15', 1764525299999, '408311.357800', 698, '875.17', '121089.490700', '0'], [1764525300000, '138.3600', '138.4000', '138.3400', '138.3500', '3621.63', 1764525359999, '501114.544200', 774, '1709.89', '236592.461900', '0'], [1764525360000, '138.3400', '138.4200', '138.3400', '138.3400', '5694.02', 1764525419999, '787957.337100', 612, '3233.63', '447503.293100', '0'], [1764525420000, '138.3500', '138.3900', '138.3400', '138.3500', '3128.59', 1764525479999, '432874.263800', 439, '1959.22', '271087.630900', '0'], [1764525480000, '138.3500', '138.4000', '138.3400', '138.3900', '2912.09', 1764525539999, '402918.771500', 447, '1062.19', '146965.886500', '0'], [1764525540000, '138.3900', '138.4100', '138.3800', '138.4100', '2623.13', 1764525599999, '363029.790100', 312, '2003.49', '277276.669900', '0'], [1764525600000, '138.4100', '138.4900', '138.3600', '138.4700', '16817.48', 1764525659999, '2327768.029800', 1664, '10135.92

In [37]:
df_raw = pd.DataFrame(data)
df_raw.columns = [
    "open_time", "open", "high", "low", "close", "volume",
    "close_time", "quote_asset_volume", "number_of_trades",
    "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume", "ignore"
]
df_raw

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
0,1764525240000,138.3600,138.3800,138.3400,138.3600,2951.15,1764525299999,408311.357800,698,875.17,121089.490700,0
1,1764525300000,138.3600,138.4000,138.3400,138.3500,3621.63,1764525359999,501114.544200,774,1709.89,236592.461900,0
2,1764525360000,138.3400,138.4200,138.3400,138.3400,5694.02,1764525419999,787957.337100,612,3233.63,447503.293100,0
3,1764525420000,138.3500,138.3900,138.3400,138.3500,3128.59,1764525479999,432874.263800,439,1959.22,271087.630900,0
4,1764525480000,138.3500,138.4000,138.3400,138.3900,2912.09,1764525539999,402918.771500,447,1062.19,146965.886500,0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1764530940000,138.0200,138.0300,137.8200,137.8300,15207.61,1764530999999,2097421.761600,1720,3040.66,419237.114200,0
96,1764531000000,137.8200,137.8300,137.7500,137.7500,12975.10,1764531059999,1787766.379100,1825,4193.82,577874.310600,0
97,1764531060000,137.7500,137.8000,137.6900,137.7600,15294.43,1764531119999,2106629.314900,1606,8102.02,1115977.597200,0
98,1764531120000,137.7600,137.7800,137.6200,137.6600,12172.74,1764531179999,1676267.019300,1710,4298.12,591952.820900,0


In [38]:
# --- DataFrame 1: Opening Data ---
df_open = df_raw[['open_time', 'open']].copy() 
df_open.columns = ['opening_time', 'opening_price']

# 1. Convert 'opening_time' from Unix timestamp (ms) to readable datetime objects
# 2. Convert 'opening_price' from string to float for numerical analysis
df_open['opening_time'] = pd.to_datetime(df_open['opening_time'], unit='ms')
df_open['opening_price'] = df_open['opening_price'].astype(float)
df_open

,opening_time,opening_price
0,2025-11-30 17:54:00,138.36
1,2025-11-30 17:55:00,138.36
2,2025-11-30 17:56:00,138.34
3,2025-11-30 17:57:00,138.35
4,2025-11-30 17:58:00,138.35
...,...,...
95,2025-11-30 19:29:00,138.02
96,2025-11-30 19:30:00,137.82
97,2025-11-30 19:31:00,137.75
98,2025-11-30 19:32:00,137.76


In [39]:
# --- DataFrame 2: Closing Data ---
df_close = df_raw[['close_time', 'close']].copy()
df_close.columns = ['closing_time', 'closing_price']

# Data Type Conversion:
# 1. Convert 'closing_time' from Unix timestamp (ms) to readable datetime objects
# 2. Convert 'closing_price' from string to float
df_close['closing_time'] = pd.to_datetime(df_close['closing_time'], unit='ms')
df_close['closing_price'] = df_close['closing_price'].astype(float)
df_close

,closing_time,closing_price
0,2025-11-30 17:54:59.999,138.36
1,2025-11-30 17:55:59.999,138.35
2,2025-11-30 17:56:59.999,138.34
3,2025-11-30 17:57:59.999,138.35
4,2025-11-30 17:58:59.999,138.39
...,...,...
95,2025-11-30 19:29:59.999,137.83
96,2025-11-30 19:30:59.999,137.75
97,2025-11-30 19:31:59.999,137.76
98,2025-11-30 19:32:59.999,137.66


In [40]:
# line chart for opening prices
fig = px.line(
    df_open,
    x='opening_time',
    y='opening_price',
    title='Line Chart of Opening Prices Over Time'
)
fig.show()

In [41]:
# line chart for closing prices
fig = px.line(
    df_close,
    x='closing_time',
    y='closing_price',
    title='Line Chart of Closing Prices Over Time'
)
fig.show()

In [42]:
# Rename time columns for clarity before merging
df_open_merge = df_open.rename(columns={'opening_time': 'time'})
df_close_merge = df_close.rename(columns={'closing_time': 'close_time'})

# Merge the two DataFrames
# CRITICAL: We merge on the index (left_index=True, right_index=True).
# We cannot merge on 'time' because the Open Time and Close Time of the same candle are different.
# By merging on the index, we ensure that the Open and Close prices for the *same candle* are aligned.
df_combined = pd.merge(
    df_open_merge[['time', 'opening_price']],
    df_close_merge[['closing_price']],
    left_index=True,
    right_index=True,
    how='inner'
)

df_combined_long = pd.melt(
    df_combined,
    id_vars=['time'],
    value_vars=['opening_price', 'closing_price'],
    var_name='Price_Type',      # New column for the label (e.g., 'opening_price')
    value_name='Price_Value'    # New column for the data (e.g., 45000)
)

df_combined_long

,time,Price_Type,Price_Value
0,2025-11-30 17:54:00,opening_price,138.36
1,2025-11-30 17:55:00,opening_price,138.36
2,2025-11-30 17:56:00,opening_price,138.34
3,2025-11-30 17:57:00,opening_price,138.35
4,2025-11-30 17:58:00,opening_price,138.35
...,...,...,...
195,2025-11-30 19:29:00,closing_price,137.83
196,2025-11-30 19:30:00,closing_price,137.75
197,2025-11-30 19:31:00,closing_price,137.76
198,2025-11-30 19:32:00,closing_price,137.66


In [43]:
fig = px.line(
    df_combined_long,
    x='time',          
    y='Price_Value',
    color='Price_Type',
    title='Opening vs. Closing Prices Over Time'
)

# Customize the labels displayed in the legend/tooltip
fig.for_each_trace(lambda t: t.update(name = t.name.replace('_', ' ').title()))

fig.update_layout(
    xaxis_title='Time',
    yaxis_title='Price',
    hovermode='x unified'
)

fig.show()